# SentinelHub Statistical API
## Outline
1. SentinelHub configuration
2. Define AOI
3. Prepare collections
   1. BYOC collections
   2. FAIRiCube collections
3. Request collections (data fusion)

## Resources
### SentinelHub Documentation (higher level)


### SentinelHub Python documentation
- Statistical API <https://sentinelhub-py.readthedocs.io/en/latest/examples/statistical_request.html>

## 1. SentinelHub configuration

In [1]:
# Configure plots for inline use in Jupyter Notebook
%matplotlib inline

import datetime as dt

# Utilities
import boto3
import dateutil
import geopandas as gpd
import matplotlib.pyplot as plt
import pandas as pd
import os
import rasterio
# Various utilities
import json
import xarray as xr
import shapely.geometry
import IPython.display
import zarr

# Sentinel Hub
from sentinelhub import (
    CRS,
    BBox,
    ByocCollection,
    ByocCollectionAdditionalData,
    ByocCollectionBand,
    ByocTile,
    DataCollection,
    DownloadFailedException,
    MimeType,
    SentinelHubBYOC,
    SentinelHubRequest,
    SentinelHubStatistical,
    SHConfig,
    bbox_to_dimensions,
    os_utils,
)

# The following is not a package. It is a file utils.py which should be in the same folder as this notebook.
# from utils import plot_image

config = SHConfig()
config.instance_id = os.environ.get("SH_INSTANCE_ID")
config.sh_client_id = os.environ.get("SH_CLIENT_ID")
config.sh_client_secret = os.environ.get("SH_CLIENT_SECRET")
config.aws_access_key_id = os.environ.get("username")
config.aws_secret_access_key = os.environ.get("password")

## helper function
def stats_to_df(stats_data):
    """Transform Statistical API response into a pandas.DataFrame"""
    df_data = []

    for single_data in stats_data["data"]:
        df_entry = {}
        is_valid_entry = True

        df_entry["interval_from"] = parse_time(single_data["interval"]["from"]).date()
        df_entry["interval_to"] = parse_time(single_data["interval"]["to"]).date()

        for output_name, output_data in single_data["outputs"].items():
            for band_name, band_values in output_data["bands"].items():
                band_stats = band_values["stats"]
                if band_stats["sampleCount"] == band_stats["noDataCount"]:
                    is_valid_entry = False
                    break

                for stat_name, value in band_stats.items():
                    col_name = f"{output_name}_{band_name}_{stat_name}"
                    if stat_name == "percentiles":
                        for perc, perc_val in value.items():
                            perc_col_name = f"{col_name}_{perc}"
                            df_entry[perc_col_name] = perc_val
                    else:
                        df_entry[col_name] = value

        if is_valid_entry:
            df_data.append(df_entry)

    return pd.DataFrame(df_data)


## 2. Define AOI and visualize it

In [2]:
x1 = 6  # degree
y1 = 49.5  # degree
x2 = 6.3 # degree
y2 = 49.7  # degree

bbox_lux = x1, y1, x2, y2

resolution = 60
lux_bbox = BBox(bbox=bbox_lux, crs=CRS.WGS84)
lux_size = bbox_to_dimensions(lux_bbox, resolution=resolution)
print(f"Image shape at {resolution} m resolution: {lux_size} pixels")

IPython.display.GeoJSON(shapely.geometry.box(*bbox_lux).__geo_interface__)

Image shape at 60 m resolution: (375, 357) pixels


<IPython.display.GeoJSON object>

## 3. Prepare collections
### 3.1 BYOC collection
- Collection name: UrbanAtlas2018_10m_raster
- CollectionId: bc4099db-f686-4e66-99a6-387a11eb2067

In [3]:
collection_id_UA = "27c95f8f-fb6b-46f2-8f84-2ce7761eeccd"
collection_name_UA = "UrbanAtlas2018_10m_raster_v1"
data_collection_UA = DataCollection.define_byoc(collection_id_UA, name=collection_name_UA, is_timeless = True)
data_collection_UA

<DataCollection.UrbanAtlas2018_10m_raster_v1: DataCollectionDefinition(
  api_id: byoc-27c95f8f-fb6b-46f2-8f84-2ce7761eeccd
  catalog_id: byoc-27c95f8f-fb6b-46f2-8f84-2ce7761eeccd
  wfs_id: byoc-27c95f8f-fb6b-46f2-8f84-2ce7761eeccd
  collection_type: BYOC
  collection_id: 27c95f8f-fb6b-46f2-8f84-2ce7761eeccd
  is_timeless: True
  has_cloud_coverage: False
)>

In [4]:
# list band names of the collection
byoc = SentinelHubBYOC(config=config)
collection = byoc.get_collection("b17f90da-1308-439e-b675-6e3a87f883de")
collection["additionalData"]

{'bands': {'B1': {'bitDepth': 16,
   'source': 'B1',
   'bandIndex': 1,
   'sampleFormat': 'UINT'}},
 'maxMetersPerPixel': 6400.0,
 'extent': {'type': 'Polygon',
  'coordinates': [[[-45.383833959, 26.085686876],
    [-45.383833959, 67.656031646],
    [53.430124419, 67.656031646],
    [53.430124419, 26.085686876],
    [-45.383833959, 26.085686876]]]},
 'hasSensingTimes': 'NO'}

In [5]:
collection_id_FUA = "20774d38-7b2e-43e9-b574-66fa2954da1c"
collection_name_FUA = "urban_audit_2021_city"
data_collection_FUA = DataCollection.define_byoc(collection_id_FUA, name=collection_name_FUA, is_timeless = True)
data_collection_FUA

<DataCollection.urban_audit_2021_city: DataCollectionDefinition(
  api_id: byoc-20774d38-7b2e-43e9-b574-66fa2954da1c
  catalog_id: byoc-20774d38-7b2e-43e9-b574-66fa2954da1c
  wfs_id: byoc-20774d38-7b2e-43e9-b574-66fa2954da1c
  collection_type: BYOC
  collection_id: 20774d38-7b2e-43e9-b574-66fa2954da1c
  is_timeless: True
  has_cloud_coverage: False
)>

In [6]:
# copy CollectionId from FAIRiCube catalog https://catalog.fairicube.eu/
collection_id_popdens = "b468089b-2627-4787-b984-89c10434f6c6"
collection_name_popdens = "Population_density"
# define collection
data_collection_popdens = DataCollection.define_byoc(collection_id_popdens, name=collection_name_popdens)
data_collection_popdens

<DataCollection.Population_density: DataCollectionDefinition(
  api_id: byoc-b468089b-2627-4787-b984-89c10434f6c6
  catalog_id: byoc-b468089b-2627-4787-b984-89c10434f6c6
  wfs_id: byoc-b468089b-2627-4787-b984-89c10434f6c6
  collection_type: BYOC
  collection_id: b468089b-2627-4787-b984-89c10434f6c6
  is_timeless: False
  has_cloud_coverage: False
)>

In [7]:
# simple evalscript to check if single request works
evalscript_test = """

//VERSION=3
function setup() {
  return {
    input: [{
    bands: [
    "B1",
    "dataMask"
    ]
    }],
    output: [{ 
    id: "UA",
        bands: 1,
        sampleType: "UINT16" // raster format will be UINT16
        },
        {
        id: "dataMask",
        bands: 1}]
    
  };
}

function evaluatePixel(sample) {
  return {
    UA: [sample.B1],
    dataMask: [sample.dataMask]
    };
}
"""

request = SentinelHubStatistical(
    aggregation=SentinelHubStatistical.aggregation(
        evalscript=evalscript_test,
        time_interval=("2021-05-01", "2021-05-01"),
        aggregation_interval="P1D",
        size=lux_size
    ),
    input_data=[SentinelHubStatistical.input_data(DataCollection.urban_audit_2021_city)],
    bbox=lux_bbox,
    config=config,
)

data = request.get_data()[0]
data

{'data': [], 'status': 'OK'}